In [1]:
import glob
import numpy as np
import os.path as path
from scipy import misc

In [3]:
import cv2

In [ ]:
images = glob.glob('trafficlight_images/*.jpg')

for image in images:
    img = cv2.imread(image)
    resizeimg = cv2.resize(img, (120, 256), interpolation=cv2.INTER_CUBIC)
    cv2.imwrite(image.split('.')[0] + '_resize.jpg', resizeimg)

In [2]:
# IMAGE_PATH should be the path to the planesnet folder
IMAGE_PATH = 'TrafficLight_Small'
file_paths = glob.glob(IMAGE_PATH + '/*resize.jpg')

# Load the images
images = [misc.imread(path) for path in file_paths]

print('Num of pictures to train: ', len(images), 'Resolution: ', images[0].shape)

/Users/gaspards/miniconda3/envs/py27_TF1.3/lib/python2.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  


('Num of pictures to train: ', 540, 'Resolution: ', (256, 120, 3))


In [4]:
images = np.asarray(images)

print(images.shape)

# Get image size
image_size = np.asarray([images.shape[1], images.shape[2], images.shape[3]])
print(image_size)

(540, 256, 120, 3)
[256 120   3]


In [5]:
# Scale
images = images / 255

# Read the labels from the filenames
n_images = images.shape[0]
labels = np.zeros(n_images)
for i in range(n_images):
    #filename = path.basename(file_paths[i])[0]
    filename = file_paths[i].split('/')[-1]
    labels[i] = int(filename[0])

In [7]:
# Split into test and training sets
TRAIN_TEST_SPLIT = 0.9

# Split at the given index
split_index = int(TRAIN_TEST_SPLIT * n_images)
shuffled_indices = np.random.permutation(n_images)
train_indices = shuffled_indices[0:split_index]
test_indices = shuffled_indices[split_index:]

In [8]:
# Split the images and the labels
x_train = images[train_indices, :, :]
y_train = labels[train_indices]
x_test = images[test_indices, :, :]
y_test = labels[test_indices]

In [9]:
## MODEL CREATION ##
# Imports
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping, TensorBoard
from sklearn.metrics import accuracy_score, f1_score
from datetime import datetime

Using TensorFlow backend.


In [10]:
# Model Hyperparamaters
N_LAYERS = 4

def cnn(size, n_layers):
    # INPUTS
    # size     - size of the input images
    # n_layers - number of layers
    # OUTPUTS
    # model    - compiled CNN

    # Define model hyperparamters
    MIN_NEURONS = 20
    MAX_NEURONS = 120
    KERNEL = (3, 3)

    # Determine the # of neurons in each convolutional layer
    steps = np.floor(MAX_NEURONS / (n_layers + 1))
    nuerons = np.arange(MIN_NEURONS, MAX_NEURONS, steps)
    nuerons = nuerons.astype(np.int32)

    # Define a model
    model = Sequential()

    # Add convolutional layers
    for i in range(0, n_layers):
        if i == 0:
            shape = (size[0], size[1], size[2])
            model.add(Conv2D(nuerons[i], KERNEL, input_shape=shape))
        else:
            model.add(Conv2D(nuerons[i], KERNEL))

        model.add(Activation('relu'))

    # Add max pooling layer with dropout
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(MAX_NEURONS))
    model.add(Activation('relu'))

    # Add output layer
    model.add(Dense(3))
    model.add(Activation('sigmoid'))

    # Compile the model
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    # Print a summary of the model
    model.summary()

    return model

In [ ]:
# Instantiate the model
model = cnn(size=image_size, n_layers=N_LAYERS)


## MODEL TRAINING ##
# Training Hyperparamters
EPOCHS = 50
BATCH_SIZE = 200

# Early stopping callback
PATIENCE = 10
early_stopping = EarlyStopping(monitor='loss', min_delta=0, patience=PATIENCE, verbose=0, mode='auto')

# TensorBoard callback
#LOG_DIRECTORY_ROOT = 'logdir'
#now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
#log_dir = "{}/run-{}/".format(LOG_DIRECTORY_ROOT, now)
#tensorboard = TensorBoard(log_dir=log_dir, write_graph=True, write_images=True)

# Place the callbacks in a list
#callbacks = [early_stopping, tensorboard]
callbacks = [early_stopping]

from sklearn.preprocessing import LabelBinarizer
label_binarizer = LabelBinarizer()
y_one_hot = label_binarizer.fit_transform(y_train)

# Train the model
model.fit(x_train, y_one_hot, epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks, verbose=0)

model.save('model.h5')

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 254, 118, 20)      560       
_________________________________________________________________
activation_1 (Activation)    (None, 254, 118, 20)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 252, 116, 44)      7964      
_________________________________________________________________
activation_2 (Activation)    (None, 252, 116, 44)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 250, 114, 68)      26996     
_________________________________________________________________
activation_3 (Activation)    (None, 250, 114, 68)      0      

In [ ]:
from keras.models import load_model

model = load_model('model.h5')

model.summary()

In [ ]:
## MODEL EVALUATION ##
# Make a prediction on the test set
test_predictions = model.predict(x_test)
test_predictions = np.round(test_predictions)
y_one_hot_test = label_binarizer.fit_transform(y_test)

In [ ]:
# Report the accuracy
accuracy = accuracy_score(y_one_hot_test, test_predictions)
print("Accuracy: " + str(accuracy))

In [ ]:
# Try inference couple samples
#test1_img = misc.imread('trafficlight_images/1_image306_resize.jpg')
test1_img = cv2.imread('trafficlight_images/1_image327_resize.jpg')
test1_img = np.asarray(test1_img) / 255

test2_img = cv2.imread('trafficlight_images/0_image296_resize.jpg')
test2_img = np.asarray(test2_img) / 255

test3_img = cv2.imread('trafficlight_images/2_image293_resize.jpg')
test3_img = np.asarray(test3_img) / 255

test = np.array([test1_img, test2_img, test3_img])

test_pred = model.predict(test)
test_pred = np.round(test_pred)

#from sklearn.preprocessing import LabelBinarizer

result = label_binarizer.inverse_transform(test_pred)
print(result)
